In [1]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')

# Launch DPS for extract_filter_atl08.py
using the rebinned 30m h5 granules

In [20]:
from os import path
import os, glob
import datetime
import pandas as pd

In [3]:
curr_dir = wk_dir = os.path.dirname(os.path.realpath('__file__'))
print(curr_dir)

/projects/icesat2_boreal/dps/alg_2-3


## Test extract_filter_atl08.py code on Test Data
We are running extract_filter_atl08.py (but *FILTERING* is turned off):

python /projects/icesat2_boreal/lib/extract_filter_atl08.py -o "/projects/test_data/test_data_30m" -i "/projects/my-private-bucket/dps_output/run_rebinning_ubuntu/master/2022/02/25/18/51/17/542274/ATL08_30m_20190721220156_03640403_005_01.h5" --no-filter-qual --do_30m

python /projects/icesat2_boreal/lib/extract_filter_atl08.py -o "/projects/test_data/test_data_30m" -i "https://maap-ops-workspace.s3.amazonaws.com.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/02/25/18/51/17/542274/ATL08_30m_20190721220156_03640403_005_01.h5" --no-filter-qual --do_30m

python lib/extract_filter_atl08.py -i "path/to/h5file" -o "path/of/out/dir" --no-filter-qual --do_30m

### Build the input list of granules
##### replace the local path with the https url

In [11]:
%%time
INPUT_ATL08_GRANULE_LIST = glob.glob("/projects/my-private-bucket/dps_output/run_rebinning_ubuntu/master/2022/**/*.h5", recursive=True)
INPUT_ATL08_GRANULE_LIST = [i.replace("/projects/my-private-bucket", "https://maap-ops-workspace.s3.amazonaws.com/lduncanson") for i in INPUT_ATL08_GRANULE_LIST]
len(INPUT_ATL08_GRANULE_LIST)
print(f"# rebinned (30m) ATL08 granules: {len(INPUT_ATL08_GRANULE_LIST)}")

# rebinned (30m) ATL08 granules: 1107
CPU times: user 307 ms, sys: 444 ms, total: 751 ms
Wall time: 5min 12s


#### Set TEST, get list

In [13]:
TEST_DPS  = True

if TEST_DPS:
    print('Running DPS on a SUBSET list of input')
    DPS_INPUT_ATL08_GRANULE_LIST = INPUT_ATL08_GRANULE_LIST[-10:]
else:
    print('Running DPS on the FULL list of input')
    DPS_INPUT_ATL08_GRANULE_LIST = INPUT_ATL08_GRANULE_LIST
    
DPS_INPUT_ATL08_GRANULE_LIST[-1]

Running DPS on a SUBSET list of input


'https://maap-ops-workspace.s3.amazonaws.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/03/01/03/56/48/796638/ATL08_30m_20190421213931_03610303_005_01.h5'

## Run a DPS job across the list

In [21]:
job_results_list = []

for i, INPUT_ATL08_GRANULE in enumerate(DPS_INPUT_ATL08_GRANULE_LIST):

    DPS_num = i+1
    IDENTIFIER = 'run_extract_filter_atl08'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'lduncanson'
    WORKER_TYPE = 'maap-dps-worker-8gb'
    
    in_param_dict = {
                        'input_file': INPUT_ATL08_GRANULE
                    }

    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='master',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )
    
    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_ATL08_GRANULE
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE
    job_results_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 100, 500, 1000, 3000, len(INPUT_ATL08_GRANULE_LIST)]:
        #print(f"DPS run #: {DPS_num} | granule name: {os.path.basename(INPUT_ATL08_GRANULE)} | job info: {submit_result}") 
        print(submit_result)
        
job_results_df = pd.concat(job_results_list)
job_results_df

{'status': 'success', 'http_status_code': 200, 'job_id': 'ea360c16-a06e-4a96-9634-9076867d3c4a', 'dps_num': 1, 'tile_num': 'https://maap-ops-workspace.s3.amazonaws.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/02/28/22/17/20/729492/ATL08_30m_20181014093633_02410102_005_01.h5', 'submit_time': datetime.datetime(2022, 3, 1, 19, 32, 7, 108405), 'dbs_job_hour': 19, 'algo_id': 'run_extract_filter_atl08_ubuntu', 'user': 'lduncanson', 'worker_type': 'maap-dps-worker-8gb'}


,status,http_status_code,job_id,dps_num,tile_num,submit_time,dbs_job_hour,algo_id,user,worker_type
0,success,200,ea360c16-a06e-4a96-9634-9076867d3c4a,1,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-01 19:32:07.108405,19,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,c8ff1fc9-45ed-47c0-9995-72931ea3b310,2,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-01 19:32:07.194752,19,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,cab5b165-5fb8-4643-bdd7-730986133fac,3,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-01 19:32:07.277363,19,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,e85fb88a-6d33-4382-8a9c-2f4492473ed8,4,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-01 19:32:07.393903,19,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,7ab8f183-c1a3-4d9b-a0e3-17e8d0b71148,5,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-01 19:32:07.529159,19,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,688e7e32-3130-4594-bc6a-0b227b4a7e10,6,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-01 19:32:07.644433,19,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,7085b5f6-c63e-486b-9b82-6b5a239add82,7,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-01 19:32:07.735843,19,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,d10d404d-50ec-452f-84a0-36c71c937527,8,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-01 19:32:07.818285,19,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,8f1ce028-253e-480e-a28e-6aa1617dcb48,9,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-01 19:32:07.938641,19,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,627f91fe-c67d-41f3-845b-69f6638913a6,10,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-01 19:32:08.146047,19,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb


In [31]:
print(f"The data frame show you submitted {len(job_results_df)} jobs. Check the returned results to see if the total returned = total submitted...")
for JOB_HOUR in [18,19,20, 21]:
    returned_results_list = glob.glob(f"/projects/my-private-bucket/dps_output/run_extract_filter_atl08_ubuntu/master/2022/03/01/{JOB_HOUR}/**/_stdout.txt", recursive=True)
    print(f"For DPS job that returned results in hour {JOB_HOUR}, # granules that ran: {len(returned_results_list)}")

The data frame show you submitted 10 jobs. Check the returned results to see if the total returned = total submitted...
For DPS job that returned results in hour 18, # granules that ran: 0
For DPS job that returned results in hour 19, # granules that ran: 14
For DPS job that returned results in hour 20, # granules that ran: 0
For DPS job that returned results in hour 21, # granules that ran: 0


In [48]:
os.system( 'python /projects/icesat2_boreal/lib/build_tindex_master.py -t ATL08 -dps_year 2021 --outdir /projects/my-public-bucket/test_output')

512